In [6]:
from bs4 import BeautifulSoup
import requests as req

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials, SpotifyOAuth
import sys

import numpy as np
import pandas as pd

from tqdm.notebook import tqdm
import time

scope = "user-library-read"
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(scope=scope))
spotify = spotipy.Spotify(auth_manager=SpotifyClientCredentials())

# Get Names of Chicago Rap Artists ([Every Noise](https://everynoise.com/))

In [2]:
def generate_beautifulSoup(url):
    Web = req.get(url)
    S = BeautifulSoup(Web.text, 'lxml')
    return(S)

def top_n_artists(S, n=10):
    
    # finding the div where all of the names are stored
    canvas = S.find('div', class_="canvas")
    
    artists = []
    for div in canvas:

        
        # the elements in canvas alternate between divs with information and blank tags, 
        # we only want to look at the ones with information
        if (len(div) > 1):

            # finding the font size
            # font size measures artist importance
            style = div.get('style')
            style_attr = style.split('; ')
            importance = '';
            for pair in style_attr:
                attr, val = pair.split(': ')
                if (attr == 'font-size'):
                    importance = val
                    
            # finding the artist name 
            onclick = div.get('onclick')
            artist_string = onclick.split(', ')[1]
            artist = artist_string.split('"')[1]
            
            # appending info
            artists.append((artist, importance))



    # sorting artist by "importance"
    artists = sorted(artists, key=lambda x: x[1], reverse=True)[0:n]
    top_names = [x[0] for x in artists]

    
    
    return(top_names)

In [3]:
# running both functions with chicagorap
S = generate_beautifulSoup('https://everynoise.com/engenremap-chicagorap.html')
artist_names = top_n_artists(S, n=10)
print(artist_names)

['Kanye West', 'Juice WRLD', 'Polo G', 'Lil Durk', 'King Von', 'Chief Keef', 'Jeremih', 'Chance the Rapper', 'G Herbo', 'Lupe Fiasco']


# Get All Tracks from Artists' Albums and their Features' Albums

In [25]:
def get_artists_from_name(artist_names, show_output=False):
    '''
    get artist information from a list of artist names
    '''
    
    artists = []

    print("Name -> Artist:")
    for name in tqdm(artist_names):
        results = spotify.search(q='artist:' + name, type='artist')
        items = results['artists']['items']
        if len(items) > 0:
            artist = items[0]
            artists.append(artist)
            if show_output: print((artist['name'], artist['uri'], artist['genres']))
            
    return artists

def get_albums_from_artist(artists, show_output=False):
    '''
    get all albums from artist without duplicates
    '''

    albums = []
    albums_set = set()
    
    print("Artist -> Album:")
    for artist in tqdm(artists):

        results = spotify.artist_albums(artist['uri'], album_type='album')

        # only add new albums
        for album in results['items']:
            all_artists = [album_artist['name'] for album_artist in album['artists']]
            album_and_artist = f"{album['name']} {' '.join(all_artists)}" # assume static artist order

            if album_and_artist not in albums_set: albums.append(album)
            albums_set.add(album_and_artist)

        # continue loading albums if there are more
        while results['next']:
            results = spotify.next(results)

            for album in results['items']:
                all_artists = [album_artist['name'] for album_artist in album['artists']]
                album_and_artist = f"{album['name']} {' '.join(all_artists)}"

                if album_and_artist not in albums_set: albums.append(album)
                albums_set.add(album_and_artist)
    
    return albums

def get_artists_and_tracks_from_albums(albums):
    '''
    get all songs from a list albums
    '''
    feature_names = []
    tracks = []
    track_set = set()
    
    print("Album -> Artists, Tracks:")
    for album in tqdm(albums):
        results = spotify.album_tracks(album['uri'])
        tracks.extend(results['items'])

        # add features to list
        for info in results['items']:
            feature_names.extend([artist['name'] for artist in info['artists']])

        # continue loading tracks if there are more
        while results['next']:
            results = spotify.next(results)
            tracks.extend(results['items'])

            for info in results['items']:
                features.extend([artist['name'] for artist in info['artists']])

    feature_names = list(set(features))
    return feature_names, tracks

In [39]:
# go from artist names -> artist -> albums -> featured artists/tracks
artists = get_artists_from_name(artist_names)
albums = get_albums_from_artist(artists)
feature_names, tracks = get_artists_and_tracks_from_albums(albums)

Name -> Artist:


  0%|          | 0/10 [00:00<?, ?it/s]

Artist -> Album:


  0%|          | 0/10 [00:00<?, ?it/s]

Album -> Artists, Tracks:


  0%|          | 0/148 [00:00<?, ?it/s]

In [40]:
# small summary
print(f"Number of Albums: {len(albums)}")
print(f"Sample: {[albums[i*len(albums)//10]['name'] for i in range(10)]}")
    
print(f'\nNumber of Featured Artists: {len(feature_names)}')
print(f"Sample: {[feature_names[i*len(feature_names)//10] for i in range(10)]}")

print(f"\nNumber of tracks: {len(tracks)}")
print(f"Sample: {[tracks[i*len(tracks)//10]['name'] for i in range(10)]}")

Number of Albums: 148
Sample: ['Donda (Deluxe)', 'The College Dropout', 'The Voice', 'Remember My Name', 'The GloFiles (Pt. 3)', 'The GloFiles (Pt. 2)', 'Big Gucci Sosa', 'Sorry 4 The Weight (Deluxe Edition)', 'Jeremih [Exclusive Edition (Explicit)]', 'Welcome to Fazoland']

Number of Featured Artists: 316
Sample: ['Jeezy', 'Snoop Dogg', 'A$AP Rocky', 'Ant Clemons', '2 Chainz', 'Tray Savage', 'Brent Faiyaz', 'Booka600', 'Ari Lennox', 'Derez De’Shon']

Number of tracks: 2310
Sample: ['Donda Chant', 'Lil Jimmy Skit', 'No Interviews', 'Check', 'NASCAR', 'Almoney', "Don't Lose no Load", 'Ten Toes Down', 'Jumpin', 'Love 2 Stunt']


In [41]:
# second round of pulling information
more_artists = get_artists_from_name(feature_names)
more_albums = get_albums_from_artist(more_artists)
more_feature_names, more_tracks = get_artists_and_tracks_from_albums(more_albums)

Name -> Artist:


  0%|          | 0/316 [00:00<?, ?it/s]

Artist -> Album:


  0%|          | 0/315 [00:00<?, ?it/s]

Album -> Artists, Tracks:


  0%|          | 0/2962 [00:00<?, ?it/s]

In [43]:
# small summary for finalized list
print(f"Number of Albums: {len(more_albums)}")
print(f"Sample: {[more_albums[i*len(more_albums)//10]['name'] for i in range(10)]}")
    
print(f'\nNumber of Featured Artists: {len(more_feature_names)}')
print(f"Sample: {[more_feature_names[i*len(more_feature_names)//10] for i in range(10)]}")

print(f"\nNumber of tracks: {len(more_tracks)}")
print(f"Sample: {[more_tracks[i*len(more_tracks)//10]['name'] for i in range(10)]}")

Number of Albums: 2962
Sample: ['The Recession 2', 'The Leek, Vol. 1', 'Live in Europe (Live in Europe, 1987)', "I Don't Feel Rehabilitated", 'flowers, the album', 'Justice (The Complete Edition)', 'Thursday (Original)', 'Fuck The World', 'Wu Victory', 'Blue Magic']

Number of Featured Artists: 316
Sample: ['Jeezy', 'Snoop Dogg', 'A$AP Rocky', 'Ant Clemons', '2 Chainz', 'Tray Savage', 'Brent Faiyaz', 'Booka600', 'Ari Lennox', 'Derez De’Shon']

Number of tracks: 46383
Sample: ['Oh Lord (feat. Tamika Mallory)', "I Don't Like", 'Eddie You Should Know Better', 'Baptized In Fire', 'Guess What', "Grammy's", 'everything you need', 'Come Thru', 'The Scroll', "Get 'em Part 2"]
